# Embed an executable in the final PDF

For now it's just a 'hello world' but hopefully I can get an executable with the C64 edition of Psychedelia embedded in the PDF. That way you can both read the PDF and also use it run and play the game like so:

```bash
./psychedelia_syndrome_exe.pdf
```

Download libcosmo to create an `ape` executable:

In [1]:
import os
!mkdir -p cosmocc
os.chdir('cosmocc')
!wget https://cosmo.zip/pub/cosmocc/cosmocc.zip
!unzip cosmocc.zip
os.chdir('..')

--2024-06-03 10:30:43--  https://cosmo.zip/pub/cosmocc/cosmocc.zip
Resolving cosmo.zip (cosmo.zip)... 34.136.86.162
Connecting to cosmo.zip (cosmo.zip)|34.136.86.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187344708 (179M) [application/zip]
Saving to: ‘cosmocc.zip’

cosmocc.zip         100%[===================>] 178.67M  6.01MB/s    in 32s     

2024-06-03 10:31:16 (5.54 MB/s) - ‘cosmocc.zip’ saved [187344708/187344708]

Archive:  cosmocc.zip
   creating: bin/
  inflating: bin/ape-x86_64.elf      
    linking: bin/x86_64-unknown-cosmo-c++filt  -> x86_64-linux-cosmo-c++filt 
    linking: bin/aarch64-unknown-cosmo-objdump  -> aarch64-linux-cosmo-objdump 
    linking: bin/x86_64-unknown-cosmo-cc  -> cosmocross 
  inflating: bin/x86_64-linux-cosmo-readelf  
  inflating: bin/ape-m1.c            
  inflating: bin/aarch64-linux-cosmo-objdump  
  inflating: bin/apelink             
    linking: bin/aarch64-linux-cosmo-cpp  -> aarch64-linux-cosmo-gcc 
  inflati

  inflating: include/third_party/zlib/crc32.inc  
  inflating: include/third_party/zlib/zlib.h  
  inflating: include/third_party/zlib/zconf.h  
  inflating: include/third_party/zlib/macros.internal.h  
  inflating: include/third_party/zlib/insert_string.inc  
  inflating: include/third_party/zlib/inftrees.internal.h  
  inflating: include/third_party/zlib/cpu_features.internal.h  
  inflating: include/third_party/zlib/inffast.internal.h  
  inflating: include/third_party/zlib/adler32_simd.inc  
  inflating: include/third_party/zlib/crc32_simd.internal.h  
  inflating: include/third_party/zlib/internal.h  
  inflating: include/third_party/zlib/inffixed.inc  
   creating: include/third_party/zlib/gz/
  inflating: include/third_party/zlib/gz/gzguts.inc  
  inflating: include/third_party/zlib/inflate.internal.h  
  inflating: include/third_party/zlib/slide_hash_simd.inc  
  inflating: include/third_party/zlib/zutil.internal.h  
  inflating: include/third_party/zlib/inffast_chunk.internal.

  inflating: include/initializer_list  
  inflating: include/ctype.h         
  inflating: include/memory.h        
   creating: include/sys/
 extracting: include/sys/cdefs.h     
  inflating: include/sys/utsname.h   
  inflating: include/sys/random.h    
  inflating: include/sys/stat.h      
  inflating: include/sys/ucontext.h  
  inflating: include/sys/reboot.h    
  inflating: include/sys/event.h     
  inflating: include/sys/types.h     
  inflating: include/sys/resource.h  
  inflating: include/sys/fcntl.h     
  inflating: include/sys/syscall.h   
  inflating: include/sys/ioctl.h     
  inflating: include/sys/signal.h    
  inflating: include/sys/prctl.h     
  inflating: include/sys/sysmacros.h  
  inflating: include/sys/procfs.h    
  inflating: include/sys/user.h      
  inflating: include/sys/poll.h      
  inflating: include/sys/msg.h       
  inflating: include/sys/ptrace.h    
  inflating: include/sys/un.h        
  inflating: include/sys/param.h     
  inflating: include/

In [2]:
c = """// hello.c
#include <stdio.h>

int main() {
  printf("I'm also a little Computer Program!\\n");
}
"""
outfile = open("hello.c",'w')
outfile.write(c)
outfile.flush()

In [3]:
!./cosmocc/bin/cosmocc -o hello hello.c

Create a placeholder file called `src/include_exe.tex` for the xetex build. This will reserve the bytes at the top of the PDF where we will inject our executable directly into the PDF file it builds.

In [35]:
EXE_OFFSET = 0x40
LINE_WIDTH = 80
stream_len = os.stat("hello").st_size - EXE_OFFSET
length_len = len(str(stream_len))

TEX = """\pdfobjcompresslevel=0
\immediate
\pdfobj
{{
<<
/Length {}
>>
stream
{}
{}
endstream
}}%
"""

# Hardcoded unfortunately, so inspect the pdf file in case it changes.
PDF_PREFIX = """%PDF-1.5
%ÐÔÅØ
1 0 obj
 << /Length {} >> stream"""

texfile = open('../src/include_exe.tex','w')

prefix_size = (len(PDF_PREFIX) - 2) + length_len
stream_len_for_prefix = stream_len - prefix_size

texfile.write(TEX.format(stream_len_for_prefix,
                        '\n'.join(["0" * (LINE_WIDTH-1)] * int(stream_len/LINE_WIDTH)),
                         "0" * (stream_len % LINE_WIDTH)
                        ))
texfile.close()

Inject our binary into the final pdf. The first 0x40 bytes are the important bit of the `ape` binary header, then we use some reserved space in the `ape` header to inject our PDF stream object containing the rest of the binary file, and then we write out the rest of the PDF file.

In [38]:
exefile = open('hello','rb')
exefile.seek(0, 0)
exe_prefix =  exefile.read(EXE_OFFSET)
exefile.seek(EXE_OFFSET + prefix_size, 0)
exe_suffix = exefile.read()

pdffile = open('../out/psychedelia_syndrome.pdf','rb')

start_offset = pdffile.read().index('stream'.encode('utf-8')) + 6
pdffile.seek(0, 0)
pdf_prefix = pdffile.read(start_offset)
end_offset = pdffile.read().index('endstream'.encode('utf-8'))
pdffile.seek(end_offset, 0)
pdf_suffix = pdffile.read()

exe_pdffile = open('../out/psychedelia_syndrome_exe.pdf','wb')
exe_pdffile.write(exe_prefix)
exe_pdffile.write(pdf_prefix)
exe_pdffile.write(exe_suffix)
exe_pdffile.write(pdf_suffix)
exe_pdffile.close()

Try it out to make sure it works!

In [39]:
!evince ../out/psychedelia_syndrome_exe.pdf

In [40]:
!chmod +x ../out/psychedelia_syndrome_exe.pdf
!../out/psychedelia_syndrome_exe.pdf

I'm also a little Computer Program!
